In [ ]:
import torch
import torch.nn.functional as F
import torch.nn as nn

In [ ]:
# hyperparameters
device = 'cuda' if torch.cuda.is_available() else 'cpu'
batch_size = 64
block_size = 256
n_embd = 32
head_size = 16
n_heads = 7
n_blocks = 10
dropout = 0.2
lr = 3e-4
max_iters = 5000
eval_iters = 200

In [ ]:
torch.manual_seed(91000)
text = open('/kaggle/input/inputs/input.txt', 'r').read()
chars = sorted(list(set(text)))
stoi = {s:i for i, s in enumerate(chars)}
itos = {i:s for s, i in stoi.items()}
vocab_size = len(chars) # 65
encode = lambda S: [stoi[s] for s in S]
decode = lambda I: ''.join([itos[i] for i in I])

In [ ]:
# data preprocessing
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data)) # 90% training split
train_split = data[:n]
val_split = data[n:]

In [ ]:
def get_batch(split):
    data = train_split if split == 'train' else val_split
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss(model):
    out = {}
    model.eval() # changing to eval mode
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train() # changing to training mode (dropout, etc. stuff turned on)
    return out

In [ ]:
# self-attention
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape # batch, time, channel
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)
        weights = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5
        weights = weights.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        weights = F.softmax(weights, dim=-1)
        weights = self.dropout(weights)
        out = weights @ v # (B, T, head_size)
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, head_size, n_heads):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(n_heads)])
        self.proj = nn.Linear(head_size*n_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [ ]:
# feed-forward
class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )
    
    def forward(self, x):
            return self.net(x)

In [ ]:
# transformer block
class Block(nn.Module):
    def __init__(self, n_embd, n_heads, head_size):
        super().__init__()
        self.sa_heads = MultiHeadAttention(head_size, n_heads)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa_heads(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [ ]:
class GPT_LM(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embd_table = nn.Embedding(vocab_size, n_embd)
        self.positional_embd_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_heads, head_size) for _ in range(n_blocks)])
        self.layernorm = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
        self.apply(self._init_weights)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, x, targets=None):
        B, T = x.shape
        tok_emb = self.token_embd_table(x)
        pos_emb = self.positional_embd_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.layernorm(x)
        logits = self.lm_head(x)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, x, max_new_tokens=700):
        for _ in range(max_new_tokens):
            context = x[:, -block_size:]
            logits, loss = self(context)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)
            x = torch.cat((x, next_token), dim=1)
        return x

In [ ]:
# defining and compiling the model
model = GPT_LM()
m = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

In [ ]:
def train(model):
    for i in range(max_iters):

        # log losses
        if i % eval_iters == 0 or i == max_iters - 1:
            loss = estimate_loss(model)
            train, val = loss['train'], loss['val']
            print(f'step: {i} --> train_loss = {train} | val_loss = {val}')
            
        # training (forward + backward pass)
        xb, yb = get_batch('train')
        logits, loss = model(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

In [ ]:
train(model)

In [ ]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))